# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can create high-quality, personalized content such as product descriptions, social media posts, and blog articles. This helps reduce content creation costs and improves content consistency.
2. **Image and Video Generation**: Generative AI can create visually appealing images and videos for marketing campaigns, advertisements, and product demos. This can help brands establish a consistent visual identity and improve their online presence.
3. **Product Design**: Generative AI can aid in product design by creating 3D models, prototypes, and mockups. This helps designers iterate faster and reduce the need for physical prototypes.
4. **Chatbots and Virtual Assistants**: Generative AI can power chatbots and virtual assistants that provide personalized customer support, answer frequently asked questions, and offer product recommendations.
5. **Predictiv

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content, such as articles, social media posts, product descriptions, and more. This can help businesses streamline their content creation process and reduce costs associated with hiring writers or freelancers.
2. **Product Design**: Generative AI can assist in the design of products by generating 3D models, prototypes, and even entire product lines. This can be particularly useful for companies that need to create complex products quickly, such as aerospace or automotive manufacturers.
3. **Marketing Automation**: Generative AI can help automate marketing efforts by generating personalized messages, emails, and social media posts based on customer behavior and preferences.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can be used to provide 24/7 customer support, helping businesses respond q

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [7]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [8]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [12]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages)
    return response.message.content

In [13]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://edwarddonner.com")

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: Generate high-quality content such as articles, social media posts, product descriptions, and more, saving time and resources for content creators.
2. **Image and Video Creation**: Use generative models to create realistic images, videos, and animations for marketing materials, advertising campaigns, and entertainment purposes.
3. **Product Design**: Utilize generative AI to design new products, such as 3D models, prototypes, and product lines, accelerating the product development process.
4. **Recommendation Systems**: Develop personalized recommendations for customers based on their preferences, behavior, and purchase history, improving customer engagement and loyalty.
5. **Chatbots and Virtual Assistants**: Create conversational interfaces that can simulate human-like conversations, providing 24/7 support and automating customer service tasks.
6. **Predictive Maintenance**: Use generative models to predict equipment failures, reducing downtime and increasing overall equipment effectiveness (OEE) in industries like manufacturing and healthcare.
7. **Data Augmentation**: Generate new data samples to augment existing datasets, improving model performance and reducing the need for expensive data collection efforts.
8. **Marketing Automation**: Automate marketing workflows by generating personalized emails, social media posts, and ad creative assets, streamlining marketing campaigns and improving ROI.
9. **Supply Chain Optimization**: Use generative AI to optimize supply chain operations, predicting demand, identifying bottlenecks, and recommending process improvements.
10. **Cybersecurity**: Develop predictive models to identify potential security threats and vulnerabilities, enabling proactive risk mitigation and improved incident response.

Industry-specific applications:

1. **Finance**: Generate personalized investment advice, automate risk assessment, and develop predictive models for credit scoring.
2. **Healthcare**: Create personalized treatment plans, predict patient outcomes, and generate medical imaging data augmentation.
3. **Manufacturing**: Optimize production processes, predict equipment failures, and design new product components.
4. **Retail**: Develop personalized shopping experiences, automate marketing campaigns, and generate visual product descriptions.
5. **Education**: Create adaptive learning materials, develop predictive models for student performance, and generate customized educational content.

These are just a few examples of the many business applications of Generative AI. As the technology continues to evolve, we can expect even more innovative use cases across various industries.